### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


TMDB API Key: 999876c67c228e3dd72575e1e96ec6e5
API key is valid!


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"

In [4]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time

    
    # Make a "GET" request and retrieve the JSON

    
    # Add a twelve second interval between queries to stay within API query limits

    
    # Try and save the reviews to the reviews_list

        # loop through the reviews["response"]["docs"] and append each review to the list

        # Print the page that was just retrieved


        # Print the page number that had no results then break from the loop
        
for page in range(20):
    query_url_with_page = f"{query_url}&page={page}"
    
    try:
        # Make a GET request to retrieve results
        response = requests.get(query_url_with_page)
        reviews = response.json()
        
        # Loop through the response and append to reviews_list
        for review in reviews["response"]["docs"]:
            review_list.append(review)
        
        # Print the query page number
        print(f"Page {page} processed.")
        
        # Wait for 12 seconds to stay within API rate limits
        time.sleep(12)
    
    except Exception as e:
        print(f"Error on page {page}: {e}")
        break



Page 0 processed.
Page 1 processed.
Page 2 processed.
Page 3 processed.
Page 4 processed.
Page 5 processed.
Page 6 processed.
Page 7 processed.
Page 8 processed.
Page 9 processed.
Page 10 processed.
Page 11 processed.
Page 12 processed.
Page 13 processed.
Page 14 processed.
Page 15 processed.
Page 16 processed.
Page 17 processed.


KeyboardInterrupt: 

In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
# first_five_reviews =json.dumps(review_list[:5], indent=4)
# first_five_reviews
print(json.dumps(review_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df = pd.json_normalize(review_list)


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
review_df["title"] = review_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
review_df["keywords"] = review_df["keywords"].apply(extract_keywords)

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = review_df["title"].to_list()

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found
# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Pausing for rate limit...")
        time.sleep(60)  # Sleep for 60 seconds to avoid hitting the rate limit

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    
    try:
        search_response = requests.get(search_url)
        search_response.raise_for_status() # Raise an HTTPError for bad responses
        search_results = search_response.json()
        
        # Include a try clause to search for the full movie details.
        # Use the except clause to print out a statement if a movie is not found.
        if 'results' in search_results and search_results['results']:
            # Get movie id
            movie_id = search_results['results'][0]['id']
            
            # Make a request for the full movie details
            movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            
            # Execute "GET" request with url
            movie_response = requests.get(movie_url)
            movie_response.raise_for_status()  # Raise an HTTPError for bad responses
            movie_details = movie_response.json()
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_details['genres']]
            
            # Extract the spoken_languages' English name into a list
            spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
            
            # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in movie_details['production_countries']]
            
            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            movie_dict = {
                "title": movie_details.get("title", ""),
                "original_title": movie_details.get("original_title", ""),
                "budget": movie_details.get("budget", 0),
                "original_language": movie_details.get("original_language", ""),
                "homepage": movie_details.get("homepage", ""),
                "overview": movie_details.get("overview", ""),
                "popularity": movie_details.get("popularity", 0),
                "runtime": movie_details.get("runtime", 0),
                "revenue": movie_details.get("revenue", 0),
                "release_date": movie_details.get("release_date", ""),
                "vote_average": movie_details.get("vote_average", 0),
                "vote_count": movie_details.get("vote_count", 0),
                "genres": genres,
                "spoken_languages": spoken_languages,
                "production_countries": production_countries
            }
            
            tmdb_movies_list.append(movie_dict)
            
            # Print out the title that was found
            print(f"Found: {title}")
        else:
            print(f"Not found: {title} - response did not contain 'results' key or was empty.")
    
    except requests.exceptions.RequestException as e:
        print(f"Request error for title '{title}': {e}")
    except KeyError as e:
        print(f"KeyError for title '{title}': {e}")
        print("Response content:", search_results)
    except Exception as e:
        print(f"Error for title '{title}': {e}")
        print("Response content:", search_results)

# Preview the first five results
print(json.dumps(tmdb_movies_list[:5], indent=4))

# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

# Display the DataFrame
tmdb_df.head()



Found: The Attachment Diaries
Not found: What’s Love Got to Do With It?’ Probably a Lo - response did not contain 'results' key or was empty.
Found: You Can Live Forever
Found: A Tourist’s Guide to Love
Found: Other People’s Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, a Love Story
Found: In From the Side
Found: After Love
Found: Alcarràs
Found: Nelly & Nadine
Found: Lady Chatterley’s Lover
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: Art of Love
Found: The Wheel
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Love & Gelato
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Fo

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",30.461,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.290,164,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",26.971,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,37.603,100,37820,2023-04-07,6.520,76,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"


In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.232,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = review_df.merge(tmdb_df, on="title", how="inner")

In [16]:
# Define columns to fix
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Define characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through columns to fix
for column in columns_to_fix:
    # Convert the column to string
    merged_df[column] = merged_df[column].astype(str)
    # Remove the unwanted characters
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the cleaned DataFrame
print(merged_df.head())


                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/you-...   
2  https://www.nytimes.com/2023/04/13/movies/one-...   
3  https://www.nytimes.com/2023/04/13/movies/the-...   
4  https://www.nytimes.com/2023/03/30/movies/a-th...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Religion comes between two girls falling in lo...  The New York Times   
2  A film adaptation of Taylor Jenkins Reid’s nov...  The New York Times   
3  There’s not much Lennon music heard in this do...  The New York Times   
4  A mesmerizing Teyana Taylor stars in A.V. Rock...  The New York Times   

                                            keywords  \
0  subject: Movies;creative_works: The Attachment...   
1  subject: Movies;creative_works: You Can Live F...   
2  subject: Movies;persons: Bracey, Lu

In [17]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

In [19]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.reset_index(drop=True)

In [20]:
# Export data to CSV without the index
merged_df.to_csv("merged_movie_data.csv", index=False)